In [409]:
class parseTitle:
    def __init__(self,titles):
        self.seplist = re.compile(":\s|-based|(?:\s|^)(?:a|an|the|in|based on|on|to|of|with|without|by|from|for|via|toward|towards|beyond|using|and|or|as|is|am|are)(?:\s|$)",re.IGNORECASE)
        self.replist = re.compile("^\s|^(?:a|an|the|in)\s|\s(?:a|an|the|in)\s|[^si](s)$|^\s$|^in$",re.IGNORECASE)
        self.pulist = re.compile("(3d reconstruction|object detection|object recognition|object categorization|6d pose|face recognition|face detection|face verification|face identification|\
pedestrian detection|open dataset|knowledge distillation|style transfer|character recognition|character region detection|text spot|\
semantic segmentation|instance segmentation|image segmenation|object tracking|image retrieval|metric learning|pre training|fine tuning|fully convolutional network|Neural Radience Field|Light field|\
action recognition|re identification|image classification|deformable matching|deformable object|non　rigid|weak supervision|transformer|autoencoder|decoder|\
knowledge transfer|domain adapation|domain adaptation|reinforcement learning|transfer learning|active learning|multi modal|cross modal|\
simultaneous localization and mapping|federated learning|point cloud|Siamese|wild|disentangl|E2E|end to end|sparse|adversarial|weakly supervised|\
weak supervision|self supervis|one shot|few shot|zero shot|image generation|video generation|question answer|captioning|\
diffusion model|human computer interaction|attention|representation learning|contrastive learning|continual learning|incremental learning|\
unsupervised|semi supervised|noisy label|soft label|deep neural network|boosting|boost|expert|spatio temporal|\
multi task learning|multiple instance learning|multi instance learning|Kullback Leibler)s*",re.IGNORECASE)
        self.malist = re.compile("[^\s-](SLAM|CNN|R CNN|RCNN|RNN|LSTM|CTC|SOM|GAN|ICA|PCA|U NET|UNET|Nerf|SVM|Bagging|MOE|\
Random forest|RF|KL divergence|DNN|GBM)s*[-\s$]")
        self.plist = re.compile(".+?\s(?:interaction|estimation|prediction|alignment|detection|recognition|classification|segmentation|\
identification|retrieval|generation|prediction|inpainting|labeling|distillation|visualizaton|reconstruction|enhancement|\
editing|idenfication|verification|authentication|restoration|tracking|matching|categorization|denoising|navigation|Attack|Grounding|amplification|Captioning|\
answering|answer|Recovery|Parsing|localization|retargeting|relocalization|Synthesis|Correction|Searching|search|compression|Rendering|DeepFake|Reading|\
benchmark|benchmarking|animation|Translation)s*",re.IGNORECASE)
        self.mlist = re.compile("\w+[-\s]*(?:clustering|graph|training|transfer|Defocus|Regression|supression|synthetic|resolution|adaptation|network|\
fusion|reasoning|topology|space|modeling|model|learning|Feature|Generalization|Detector|\
classifier|recognizer|minimization|maxmization|Expression|label|Descriptor|extractor|sparcification|regularization|quantization|coarse to fine)s*|\
Independent Component",re.IGNORECASE)
        #llist = re.compile("",re.IGNORECASE)
        self.tlist = re.compile(".*(?:3D|rgbd|video|frame|dataset|camera|infrared|depth|lider|3D|Text to Image|imagenet|wordnet)[-s]*",re.IGNORECASE)
        self.titles = titles
        self.wordhist = {}

    def wordfreq(self):
        for title in self.titles:
            #print(title)
            sepwords = self.seplist.split(title)
            bMatch = 0
            for item in sepwords:
                item = self.replist.sub("", item)
                item = re.sub( r'-' , " ", item )
                #####
                words = self.pulist.findall(item)
                if words:
                    bMatch = 1
                    for word in words:
                        if word in self.wordhist:
                            self.wordhist[word] += 1
                        else:
                            self.wordhist[word] = 1
                if bMatch:
                    continue
                #####
                words = self.malist.findall(item)
                if words:
                    bMatch = 1
                    for word in words:
                        if word in self.wordhist:
                            self.wordhist[word] += 1
                        else:
                            self.wordhist[word] = 1
                if bMatch:
                    continue
                #####
                words = self.plist.findall(item)
                if words:
                    bMatch = 1
                    for word in words:
                        if word in self.wordhist:
                            self.wordhist[word] += 1
                        else:
                            self.wordhist[word] = 1
                if bMatch:
                    continue
                #####
                words = self.mlist.findall(item)
                if words:
                    bMatch = 1
                    for word in words:
                        if word in self.wordhist:
                            self.wordhist[word] += 1
                        else:
                            self.wordhist[word] = 1
                if bMatch:
                    continue
                #####
                words = self.tlist.findall(item)
                if words:
                    bMatch = 1
                    for word in words:
                        if word in self.wordhist:
                            self.wordhist[word] += 1
                        else:
                            self.wordhist[word] = 1
                if bMatch:
                    continue

                #if bMatch == 0 :
                #    print(item)

        stwordhist = sorted( self.wordhist.items(), key = lambda x:-x[1] )
        return stwordhist
    

In [410]:
class parse_ATlist:
    def __init__(self, titles, authors):
        self.num_papers = len(titles)
        self.title_list = titles
        self.author_list = authors
        self.sjppat="^([AIUEO]|[BGKMNPR][aiueoAIUEO]|[BGKMNPR][yY][auoAUO]|D[aeoAEO]|S[aueoAUEO]|S[hH][aiouAIOU]|T[aeoAEO]|T[sS][uU]|Z[aueoAUEO]|Y[auoAUO]|\
H[aieoAIEO]|HY[auoAUO]|J[aiuoAIUO]|J[y][auoAUO]|C[hH][aiuoAIUO]|D[aeoAEO]|W[aA]|F[uU])\
([\s'nmhtkNMHTK]|[aiueoAIUEO]|([bgkmnprBGKMNPR]|[kK][kK]|[mM][mM]|[nN][nN]|[pP][pP])[aiueoAIUEO]|([bgkmnprBGKMNPR]|[kK][kK]|[mM][mM]|[nN][nN]|\
[pP][pP])[yY][auoAUO]|[dD][aeoAEO]|([sS]|[sS][sS])[aueoAUEO]|([sS]|[sS][sS])[hH][aiouAIOU]\
|([tT]|[tT][tT])[aeoAEO]|([tT]|[tT][tT])[sS][uU]|[zZ][aueoAUEO]|[yY][auoAUO]|[hH][aieoAIEO]|[hH][yY][auoAUO]|[jJ][aiuoAIUO]|[jJ][y][auoAUO]|([cC]|[cC][cC])[hH][aiuoAIUO]|[dD][aeoAEO]|[wW][aA]|[fF][uU])*?$"
        self.snjpwpat="[^n]'|-|[QXVqxv]|[áčçøåÅÇØ]|[^aiueontkh]\s|[^aiueontkh]$|nn\s|nn$|\
[aA]er|ahn|[bBjJ]iao|[bB]odo|([cC]h|[bdjBDJ])ae|[cC]hoe|[eE]ui|[eE]un|eau|[fF][ieo]|[hH]ao|[hH]oai|[hH]yun|[hH]yuk|[yY]oun|guk|[jJ]ia|[hH]ua|Niu|[jJ]ie[^in]|[hkgHKG]yun|nea|\
[sS]hui|rian|[sSgG]uo|[sS]eo[nk]|[sS]han|[sS]eung|[sS]huai|uan|\
([fF][yY]u)an$|\
[rsy]ani|rini|[rR]oha|gudo|[zZ]ere|[kK]ede|[wW]ah|[aA]hn|\
[bdfghjklmpqrvwxyzBCDFGHJKLMPQRTVWXYZ][bcdfgjklmpqrtvwxz]|c[^h]|[tT][^aeos]|s[^aueoh]|[sS]h[^aiuo]|[tT]s[^u]|[in]h$|[jJ]as"
        self.snjppat="\
(([JN]|Sh|Ch)i|Bin|[GKd]e|Bo|Hai|[TH]ao|Bei|Ku[in]|\
Akanksha|Amanda|Ameya|Amit|Anita|Anki|Ankita|Anton|Antonio|Anuja|Aran|Arya|Arun|Aida|Aren|Arezou|Aria|Ata|\
Bao|Babak|Bee|Beibei|Benjamin|Bodo|Burak|Bukun|\
Chaehun|Chajin|Chanjin|Chau|Chiara|Da|Dakai|Dami[ae]n|Dann*a|Dario|Debora|Depanshu|Dohwan|Dominik|Dongoh|Doreen|Doron|\
Erik|Ehsan|Eugene|Gyumin|Han|Hanbin|Hani|Hanna|Hannah|Hasan|Heiko|Hogun|\
Inkyu|Itai|Jamie|Jan|Jane|Janne|Jason|Jimin|Jirak|Jose|John|Joao|Joni|Junni|Juho|Juhee|Junho|Junhyuk|Kaan|Kannan|Kanika|Karan|Kate|Kenan|Kenneth|Kiana|Kien|Koushik|Leda|Lorenzo|\
Maatei|Madonna|Man|Manikanta|Marion|Matan|Mateja|Matt|Mazen|Meena|Meike|Mihai|Minho*|Minji|Min[jJ]oon|Min[jJ]un*|Minsu|Minkyo|Mubarak|Mohi|Moon|Nageen|Namo|Nikita|Nirat|\
Oren|Ori|Oladayo|Pere|Pu|Ramana|Rami|Renshuai|Reza|Ruben|Rohit|Roi|Ruwan|Ruhi|Ryan|Sachin|Sage|Saikat|Sandaru|Sami|Samitha|Samira|Sarah|Sayan|Sean|Sen|Shasha|Shayak|Shayan|Shai|Shichao|Shujon|Soomin|Suman|Suren|Sunita|Taha|Taman|Tanaya|Tapani|Tarasha|\
Udana|Wah|Yani|Yanrui|Yasaman|Yohan|Yuanbiao|Yue|Yun|Yuru|Yunhao|Zarena|Zarana|Zerui|Zeyu)\s|\
((Ch|N)a|(Ch|Sh|[YSHFOLGMB])[uo]|[YSHFOLGMKZ]e|[EH]o|An|([TYHGW]|[BDL]|Shu)ai|[BGHYMR]ao|([YRG]u|Ch|[LYHTWGNM])an|[BNJ]i|Jia|[JM]iao|[GS]ui|[YS]un|[KGZ]uo|[LM]ei|Heo|[YH]ou|Choi|Yoo|Yoon|\
Achan|Arora|Ahuja|Aksak|Akshara|Amani|Amini|Amit|Amin|Antebi|\
Badea|Basu|Bilen|Bahadori|Bandara|Ben|Benjamin|Bera|Bian|Bin|Bok|Boneh|Bouman|Bui|Choo|Desai|Dami[ae]n|Damaine|Do|Eak|Egiazarian|Emonet|Esen|Gat|Gia|Gu|Guha|Guhan|\
Ha|Hakimi|Hansen|Heidari|Himayat|Huh|Hui|Huo|Ho|Ie|Imani|Jagadeesan|Jain|Jaipuria|Jamnik|Jansen|Jayasuriya|Juge|John|Johnson|Joo|Joshi|Ju|Junge|\
Kahn|Kahnsari|Kee|Khurana|Ki|Kiani|Kijak|Kishore|Kunze|\
Laina|Ma|Madan|Maman|Mania|Mangoubi|Mannone|Masera|Mason|Masoomi|Mazaheri|Min|Mingyu|Miolane|Memari|Men|Mohan|Mok|Monteiro|Moore|\
Narayan|Narayanan|Neeman|Nayak|Nabi|Naik|Natarajan|Nauta|Nizan|Neumann|Ngo|\
Oh|Ohana|Oza|Pai|Pajarinen|Pan|Pei|Podee|Popa|Poranne|Pore|Poria|Potapenko|Pu|Rai|Raja|Rajan|Rajawat|Ramamohanarao|Ramyaa|Rane|Rau|Rakshit|Razazadeh|Razani|Rohani|Ren|Riba|Ridao|Roh|Ruta|Ryu|\
Sabo|Saria|Saenko|Saha|Sain|Saini|Samano|Samara.+?|Sani|Saran|Sarawagi|Shabanian|Shah|Shayani|Shi|Shin|Shou|Sohn|Souza|Suh|Sundararaman|Suri|Suin|Tandon|Tonekabori|\
Wanchoo|Watson|Yaesoubi|Yogatama|Yoran|Yue|Yu[ae]n|Zada|Zabih|Zadeh|Zaharia|Zou)$|\
Jin\sJin|Ran\sTao|Ran\sBen|Yuhe\sJin|Jin\sTao|Yu\sTao|Jan\sKotera|Ken\sMai|Meina\sKan|Jinko\sKanno|Rui\sTao|^.+?\s.+?\s.+?$|\
Joon|Hee" 
        self.dlm=",\s"

    def isJP(self,name):
        # extract JP name
        rmat = re.search(self.sjppat,name)
        rstop = re.search(self.snjpwpat,name)
        rnstop = re.search(self.snjppat,name)
        if rmat and not rstop and not rnstop: # Japanese-like list and Non-Japanese name filtering
            return True
        else:
            return False

    def selectJP(self):
        jpauthor = []
        jptitle = []
        jppaper = 0
        
        for ii in range(0,self.num_papers):
            ttl = self.title_list[ii]
            jpp = False
            for au in self.author_list[ii]:
                # print(au)
                if self.isJP(au):
                    # print(au)
                    jpauthor.append(au)
                    jptitle.append(ttl)
                    jpp = True
            if jpp:
                jppaper += 1
                
        return jppaper, jptitle, jpauthor



In [411]:
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import requests
from bs4 import BeautifulSoup
import time
import datetime
import re
import os

class parse_url:
    def __init__(self, url):
        self.url = url
        print('\n==================')
        print(url)
        yr = re.search('[0-9][0-9][0-9][0-9]',url)
        print(yr[0])
        if yr[0] == '2018':
            self.oralheld = True
            self.slheld = False
            self.posterheld = True
            self.oralpt = 'accepted-oral-papers'
            self.posterpt = 'accepted-poster-papers'
            self.slpt = ''
        elif yr[0] == '2019':
            self.oralheld = True
            self.slheld = False
            self.posterheld = True
            self.oralpt = 'oral-presentations'
            self.posterpt = 'poster-presentations'
            self.slpt = ''
        elif yr[0] == '2020':
            self.oralheld = True
            self.slheld = True
            self.posterheld = True
            self.oralpt = 'oral-presentations'
            self.posterpt = 'poster-presentations'
            self.slpt = 'spotlight-presentations'
        elif yr[0] == '2021':
            self.oralheld = True
            self.slheld = True
            self.posterheld = True
            self.oralpt = 'oral-presentations'
            self.posterpt = 'poster-presentations'
            self.slpt = 'spotlight-presentations'
        elif yr[0] == '2022':
            self.oralheld = True
            self.slheld = True
            self.posterheld = True
            self.oralpt = 'oral-submissions'
            self.posterpt = 'poster-submissions'
            self.slpt = 'spotlight-submissions'
        
    def selenium(self):
        time_s = time.time()

        fname = re.sub("[\:\/\.]","_",self.url) + ".txt"
        is_file = os.path.isfile(fname)
        if is_file:
            with open(fname, encoding='utf-8') as f:
                bs = BeautifulSoup(f.read(), 'html.parser')
        else:
            bs_oral = []
            bs_sl = []
            bs_poster = []
            opt = Options()
            pt = 8

            oralpt = self.oralpt
            slpt = self.slpt
            posterpt = self.posterpt
            oralheld = self.oralheld
            slheld = self.slheld
            posterheld = self.posterheld

            url_oral = self.url + '#' + oralpt
            url_sl = self.url + '#' + slpt
            url_poster = self.url + '#' + posterpt

            #opt.add_argument("--headless") # headless mode

            #oral
            if oralheld:
                print('oral...')
                try:
                    driver = webdriver.Chrome(options = opt)
                    driver.get(url_oral)
                    time.sleep(pt)
                    bs_oral.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))
                    #print(bs_oral[0].prettify())
                    #with open(fname, mode='w') as f:
                    #    f.write(str(bs_oral[0].prettify()))
                    a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+oralpt+"]"))
                    elems = a_item.find_elements(By.TAG_NAME, ("nav"))
                    if len(elems) != 0:
                        a_items = elems[0]
                        a_items = a_item.find_elements(By.CLASS_NAME, ("right-arrow"))
                        max_page = int(a_items[1].get_attribute("data-page-number"))
                        print(max_page,' pages')
                        #print(1)
                        print(end='#')
                        for ii in range(1,max_page):
                            #print(ii+1)
                            print(end='#')
                            a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+oralpt+"]"))
                            a_item = a_item.find_element(By.TAG_NAME, ("nav"))
                            a_item = a_item.find_element(By.LINK_TEXT, str(ii+1))
                            a_item.click()
                            time.sleep(pt)
                            bs_oral.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))
                except driver.exceptions.RequestException as e:
                    print("Error: ",e)
                finally:
                    driver.quit()
            else:
                bs_oral = None


            #spotlight
            if slheld:
                print('spotlight...')
                try:
                    driver = webdriver.Chrome(options = opt)
                    driver.get(url_sl)
                    time.sleep(pt)

                    bs_sl.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))

                    a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+slpt+"]"))
                    elems = a_item.find_elements(By.TAG_NAME, ("nav"))
                    if len(elems) != 0:
                        a_items = elems[0]
                        a_items = a_item.find_elements(By.CLASS_NAME, ("right-arrow"))
                        max_page = int(a_items[1].get_attribute("data-page-number"))
                        print(max_page,' pages')
                        #print(1)
                        print(end='#')
                        for ii in range(1,max_page):
                            #print(ii+1)
                            print(end='#')
                            a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+slpt+"]"))
                            a_item = a_item.find_element(By.TAG_NAME, ("nav"))
                            a_item = a_item.find_element(By.LINK_TEXT, (str(ii+1)))
                            a_item.click()
                            time.sleep(pt)
                            bs_sl.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))
                except driver.exceptions.RequestException as e:
                    print("Error: ",e)
                finally:
                    driver.quit()
            else:
                bs_sl = None

            #poster
            if posterheld:
                print('poster...')
                try:
                    driver = webdriver.Chrome(options = opt)
                    driver.get(url_poster)
                    time.sleep(pt)
                    bs_poster.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))
                    #print(driver.page_source)
                    a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+posterpt+"]"))
                    elems = a_item.find_elements(By.TAG_NAME, ("nav"))
                    if len(elems) != 0:
                        a_items = elems[0]
                        a_items = a_item.find_elements(By.CLASS_NAME, ("right-arrow"))
                        max_page = int(a_items[1].get_attribute("data-page-number"))
                        print(max_page,' pages')
                        #print(1)
                        print(end='#')
                        for ii in range(1,max_page):
                            #print(ii+1)
                            print(end='#')
                            a_item = driver.find_element(By.CSS_SELECTOR, ("div[id="+posterpt+"]"))
                            a_item = a_item.find_element(By.TAG_NAME, ("nav"))
                            a_item = a_item.find_element(By.LINK_TEXT, str(ii+1))
                            a_item.click()
                            time.sleep(pt)
                            bs_poster.append(BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser'))
                except driver.exceptions.RequestException as e:
                    print("Error: ",e)
                finally:
                    driver.quit()
            else:
                bs_poster = None

        time_e = time.time()
        etime = time_e - time_s
        print("elapsed time: {0:.2f}".format(etime))
        return bs_oral, bs_sl, bs_poster
            
    def beautifulsoup(self):
        try:
            response = requests.get(self.url)
        except requests.exceptions.RequestException as e:
            print("Error: ",e)

        bs = BeautifulSoup(response.text, 'html.parser')
        return bs



In [412]:
def ICLR(url):
    pu = parse_url(url)
    bs_orals, bs_sls, bs_posters = pu.selenium()
    
    #a aria-controls="oral-submissions"
    title_list = []
    author_list = []

    #-----------
    numoral = 0
    if bs_orals:
        for ii in range(0,len(bs_orals)):
            oralblock = bs_orals[ii].find("div",{"id":pu.oralpt})
            tts = oralblock.find_all("h4")
            aus = oralblock.find_all("div",{"class":"note-authors"})

            numoral += len(tts)
            for jj in range(0,len(tts)):
                title_list.append(tts[jj].get_text().strip())
                author_list.append(aus[jj].get_text().strip().split(', '))
        print(numoral)
            
    #-----------
    numsl = 0
    if bs_sls:
        for ii in range(0,len(bs_sls)):
            slblock = bs_sls[ii].find("div",{"id":pu.slpt})
            tts = slblock.find_all("h4")
            aus = slblock.find_all("div",{"class":"note-authors"})

            numsl += len(tts)
            for jj in range(0,len(tts)):
                title_list.append(tts[jj].get_text().strip())
                author_list.append(aus[jj].get_text().strip().split(', '))
        print(numsl)
    
    #-----------
    numposter = 0
    if bs_posters:
        for ii in range(0,len(bs_posters)):
            posterblock = bs_posters[ii].find("div",{"id":pu.posterpt})
            tts = posterblock.find_all("h4")
            aus = posterblock.find_all("div",{"class":"note-authors"})

            numposter += len(tts)
            for jj in range(0,len(tts)):
                title_list.append(tts[jj].get_text().strip())
                author_list.append(aus[jj].get_text().strip().split(', '))
        print(numposter)
    
    #print(bs.prettify())
    
    return title_list, author_list, numoral, numsl, numposter

In [413]:
urlpre = 'https://openreview.net/group?id=ICLR.cc/'
urlpost = '/Conference'

since = 2019
until = 2022

ttlist = {}
aulist = {}
numor = {}
numsl = {}
numpo = {}
numppr = {}
jpppr = {}
jptitles = {}
jpauthors = {}
hist = {}

for ii in range(since,until+1,1):
    yr = str(ii)
    url = urlpre+yr+urlpost
    
    ttlist[yr], aulist[yr], numor[yr], numsl[yr], numpo[yr] = ICLR(url)
    pAT = parse_ATlist(ttlist[yr], aulist[yr])
    jpppr[yr],jptitles[yr],jpauthors[yr] = pAT.selectJP()
    numppr[yr] = len(ttlist[yr])

    #print('==============')
    #print('RESULT')
    #print('numppr\t{0:}\t numjpppr\t{1:}'.format(numppr,jpppr[yr]))
    #print('==============')
    #for jj in range(0,len(jptitles[yr])):
    #    print('{0:}\t{1:}'.format(jptitles[yr][jj],jpauthors[yr][jj]))

    for author in jpauthors[yr]:
        if yr in hist.keys():
            if author in hist[yr].keys():
                hist[yr][author] += 1
            else:
                hist[yr][author] = 1
        else:
            hist[yr] = {author:1}

    print('---------------------')
    print('{0}\tnum papers\t{1}\tnum papers (JPN)\t{2}'.format(yr,numppr[yr],jpppr[yr]))
    print('---------------------')
    for jj in range(0,len(jpauthors[yr])):
        print('{0:}\t{1:}'.format(jpauthors[yr][jj],jptitles[yr][jj]))
    for author in hist[yr].keys():
        print('{0:}\t{1:}'.format(author,hist[yr][author]))

minpub = 2
oahist = {}
for ii in range(since,until,1):
    sthist = sorted( hist[str(ii)].items(), key = lambda x:x[1], reverse = True )
    print('---------------------')
    print(str(ii))
    print('---------------------')
    for jj in range(0,len(sthist),1):
        print("{0:}\t{1:}".format(sthist[jj][0], sthist[jj][1]))
    for jj in range(0,len(sthist),1):
        #print("{0:}\t{1:}".format(sthist[jj][0], sthist[jj][1]))
        if sthist[jj][0] in oahist:
            oahist[sthist[jj][0]] += sthist[jj][1]
        else:
            oahist[sthist[jj][0]] = sthist[jj][1]

print('---------------------')
sthist = sorted( oahist.items(), key = lambda x:x[1], reverse = True )
for ii in range(0,len(sthist),1):
    if sthist[ii][1] >= minpub:
        print("{0:}\t{1:}".format(sthist[ii][0], sthist[ii][1]))  


https://openreview.net/group?id=ICLR.cc/2019/Conference
2019
oral...
poster...
elapsed time: 26.71
24
478
---------------------
2019	num papers	502	num papers (JPN)	11
---------------------
Akihiro Matsukawa	Do Deep Generative Models Know What They Don't Know?
Shoichiro Yamaguchi	DISTRIBUTIONAL CONCAVITY REGULARIZATION FOR GANS
Masanori Koyama	DISTRIBUTIONAL CONCAVITY REGULARIZATION FOR GANS
Heiga Zen	Hierarchical Generative Modeling for Controllable Speech Synthesis
Taiji Suzuki	Adaptivity of deep ReLU network for learning in Besov and mixed smooth Besov spaces: optimal rate and curse of dimensionality
Sho Yaida	Fluctuation-dissipation relations for stochastic gradient descent
Masahiro Kato	Learning from Positive and Unlabeled Data with a Selection Bias
Takeshi Teshima	Learning from Positive and Unlabeled Data with a Selection Bias
Junya Honda	Learning from Positive and Unlabeled Data with a Selection Bias
Masashi Sugiyama	On the Minimal Supervision for Training Any Binary Classifier